## 整理MHA、MQA、GQA和MLA的区别
- 整体对比图
![mha-mqa-gqa](../../images/mha-mqa-gqa-1.png)
- MHA多头自注意力
![mha-mqa-gqa](../../images/mha-mqa-gqa-2.png)
- GQA（Group Query Attention）的优点：效果损失小，推理的时候可以加速（来自于kvcache小，内存取数少）。
- 仔细阅读 MHA, MQA 和 GQA的区别，就会发现 MHA 和 MQA 都是 GQA 的特殊表达形式
- 三者可以用同一套代码，只需要修改【GQA】代码里面的 nums_key_value_head 参数就可
- nums_key_value_head 设置等于 1 就是 MQA
- nums_key_value_head 设置等于 nums_head 就是 MHA

## 1. multi-head self-attention 实现
也可以直接由 GQA 中修改参数得到。但是本代码更完整一些

In [7]:
import math
import torch
import torch.nn as nn

class MultiHeadAttention(nn.Module):
    def __init__(self, hidden_dim, nums_head)->None:
        super().__init__()
        self.head_dim = hidden_dim // nums_head
        self.nums_head = nums_head
        self.hidden_dim  = hidden_dim
        # 一般默认有 bias，需要时刻主意，hidden_dim = head_dim * nums_head，所以最终是可以算成是 n 个矩阵
        
        self.q_proj = nn.Linear(hidden_dim, hidden_dim)
        self.k_proj = nn.Linear(hidden_dim, hidden_dim)
        self.v_proj = nn.Linear(hidden_dim, hidden_dim)
        
        self.att_dropout = nn.Dropout(0.1)
        # 输出时候的 proj
        self.o_proj = nn.Linear(hidden_dim, hidden_dim)
    def forward(self, X, attention_mask=None):
        # 需要在 mask 之前 masked_fill
        # X shape is (batch, seq, hidden_dim)
        # attention_mask shape is (batch, seq)

        batch_size, seq_len, _ = X.size()

        Q = self.q_proj(X)
        K = self.k_proj(X)
        V = self.v_proj(X)

        # shape 变成 （batch_size, num_head, seq_len, head_dim）
        q_state = Q.view(batch_size, seq_len, self.nums_head, self.head_dim).permute(
            0, 2, 1, 3
        )
        k_state = K.view(batch_size, seq_len, self.nums_head, self.head_dim).transpose(
            1, 2
        )
        v_state = V.view(batch_size, seq_len, self.nums_head, self.head_dim).transpose(
            1, 2
        )
        # 主意这里需要用 head_dim，而不是 hidden_dim
        attention_weight = (
            q_state @ k_state.transpose(-1, -2) / math.sqrt(self.head_dim)
        )
        print(type(attention_mask))
        if attention_mask is not None:
            attention_weight = attention_weight.masked_fill(
                attention_mask == 0, float("-1e20")
            )

        # 第四个维度 softmax
        attention_weight = torch.softmax(attention_weight, dim=3)
        print(attention_weight)

        attention_weight = self.att_dropout(attention_weight)
        output_mid = attention_weight @ v_state

        # 重新变成 (batch, seq_len, num_head, head_dim)
        # 这里的 contiguous() 是相当于返回一个连续内存的 tensor，一般用了 permute/tranpose 都要这么操作
        # 如果后面用 Reshape 就可以不用这个 contiguous()，因为 view 只能在连续内存中操作
        output_mid = output_mid.transpose(1, 2).contiguous()

        # 变成 (batch, seq, hidden_dim),
        output = output_mid.view(batch_size, seq_len, -1)
        output = self.o_proj(output)
        return output


attention_mask = (
    torch.tensor(
        [
            [0, 1],
            [0, 0],
            [1, 0],
        ]
    )
    .unsqueeze(1)
    .unsqueeze(2)
    .expand(3, 8, 2, 2)
)

x = torch.rand(3, 2, 128)
net = MultiHeadAttention(128, 8)
net(x, attention_mask).shape
       

<class 'torch.Tensor'>
tensor([[[[0.0000, 1.0000],
          [0.0000, 1.0000]],

         [[0.0000, 1.0000],
          [0.0000, 1.0000]],

         [[0.0000, 1.0000],
          [0.0000, 1.0000]],

         [[0.0000, 1.0000],
          [0.0000, 1.0000]],

         [[0.0000, 1.0000],
          [0.0000, 1.0000]],

         [[0.0000, 1.0000],
          [0.0000, 1.0000]],

         [[0.0000, 1.0000],
          [0.0000, 1.0000]],

         [[0.0000, 1.0000],
          [0.0000, 1.0000]]],


        [[[0.5000, 0.5000],
          [0.5000, 0.5000]],

         [[0.5000, 0.5000],
          [0.5000, 0.5000]],

         [[0.5000, 0.5000],
          [0.5000, 0.5000]],

         [[0.5000, 0.5000],
          [0.5000, 0.5000]],

         [[0.5000, 0.5000],
          [0.5000, 0.5000]],

         [[0.5000, 0.5000],
          [0.5000, 0.5000]],

         [[0.5000, 0.5000],
          [0.5000, 0.5000]],

         [[0.5000, 0.5000],
          [0.5000, 0.5000]]],


        [[[1.0000, 0.0000],
          [1.0000

torch.Size([3, 2, 128])

## 2. Group Query Attention实现

In [ ]:
import torch
import torch.nn as nn
import math

# 忽略了 attention_mask, attention_dropout; 
class GroupQueryAttention(nn.Module):
    def __init__(self, hidden_dim, nums_head, nums_key_value_head):
        super().__init__()
        assert hidden_dim % nums_head == 0 # 可以整除
        assert nums_head % nums_key_value_head == 0  # N 个 query head 为一组

        self.hidden_dim = hidden_dim
        self.nums_head = nums_head
        self.nums_key_value_head = nums_key_value_head
        self.head_dim = hidden_dim // nums_head

        # 初始化 qkv o
        self.q_proj = nn.Linear(hidden_dim, nums_head * self.head_dim)  # out feature_size (nums_head * head_dim)
        # k v out shape (nums_key_value_head * head_dim)
        self.k_proj = nn.Linear(hidden_dim, nums_key_value_head * self.head_dim)
        self.v_proj = nn.Linear(hidden_dim, nums_key_value_head * self.head_dim)

        self.o_proj = nn.Linear(hidden_dim, hidden_dim) # input_size nums_head * head_dim

    def forward(self, X, attention_mask=None):
        # X shape (batch, seq, hidden_dim)
        batch_size, seq, _ = X.size()

        # qkv projection
        q = self.q_proj(X)  # （batch, seq, hidden_dim)
        k = self.k_proj(X)
        v = self.v_proj(X) 

        # attention_weight 目标shape 是 (batch, nums_head, seq, seq)
        q = q.view(batch_size, seq, self.nums_head, self.head_dim)
        k = k.view(batch_size, seq, self.nums_key_value_head, self.head_dim)
        v = v.view(batch_size, seq, self.nums_key_value_head, self.head_dim)

        # 关注: nums_head 和 nums_key_value_head 的关系
        q = q.transpose(1, 2) # (b, nums_head, seq, head_dim)
        k = k.transpose(1, 2) # (b, nums_key_value_head, seq, head_dim)
        v = v.transpose(1, 2)  # (b, nums_key_value_head, seq, head_dim)

        # k v repeat； （广播操作）
        k = k.repeat_interleave(self.nums_head // self.nums_key_value_head, dim=1)
        v = v.repeat_interleave(self.nums_head // self.nums_key_value_head, dim=1)

        attention_score = (q @ k.transpose(2, 3)) / math.sqrt(self.head_dim)

        attention_weight = torch.softmax(attention_score, dim=-1)
        # （attention_mask 忽略） # 可以看前面的视频

        output = attention_weight @ v  # (b, nums_head, seq, head_dim)

        # output projection 变成 (b, seq, hidden_dim)
        output = output.transpose(1, 2).contiguous()
        final_output = self.o_proj(output.view(batch_size, seq, -1))

        return final_output

# 测试
x = torch.rand(3, 2, 128)
net = GroupQueryAttention(128, 8, 4)
net(x).shape

## 3.Multi Query Attention
由于 MQA 是 GQA 的一种特殊形式，因此只要在参数设置的时候将 nums_key_value_head = 1 就是 Multi Query Self-Attention。

## 补充:GroupQueryAttention关键代码
~~~python
k = k.repeat_interleave(self.nums_head // self.nums_key_value_head, dim=1)
v = v.repeat_interleave(self.nums_head // self.nums_key_value_head, dim=1)
~~~
这两行代码实现了 GQA 中的"分组共享"机制，具体解释如下：

1. 背景 ：
   
   - 在这个实现中， nums_head 是查询头的总数量
   - nums_key_value_head 是键值头的数量（小于或等于查询头数量）
   - GQA 的核心思想是让多个查询头共享同一个键值头
2. 计算逻辑 ：
   
   - self.nums_head // self.nums_key_value_head 计算每个键值头需要被多少个查询头共享
   - 例如，如果 nums_head=8 且 nums_key_value_head=4 ，则每个键值头被2个查询头共享
3. repeat_interleave 操作 ：
   
   - 这个函数沿着指定维度（这里是 dim=1，即头的维度）重复张量中的元素
   - 它不是简单地复制整个张量，而是交错地重复每个元素指定次数
   - 在这个例子中，如果 k 的形状是 [batch_size, 4, seq_len, head_dim] ，操作后会变成 [batch_size, 8, seq_len, head_dim]
## 实际效果
假设 nums_head=8 且 nums_key_value_head=4 ，则：

- 原始 k 和 v 的形状： [batch_size, 4, seq_len, head_dim]
- 重复后的形状： [batch_size, 8, seq_len, head_dim]
重复的模式如下（用索引表示）：

- 原始 k/v 的头索引：[0, 1, 2, 3]
- 重复后的头索引：[0, 0, 1, 1, 2, 2, 3, 3]
这意味着：

- 查询头0和1共享键值头0
- 查询头2和3共享键值头1
- 查询头4和5共享键值头2
- 查询头6和7共享键值头3
## 为什么这样设计
1. 内存效率 ：减少了键值矩阵的参数量和计算量
2. 推理加速 ：在推理时，KV缓存的大小减小，提高了内存效率和计算速度
3. 灵活性 ：通过调整 nums_key_value_head 参数，可以实现从 MHA（当 nums_key_value_head = nums_head ）到 MQA（当 nums_key_value_head = 1 ）的连续过渡
这种设计在大型语言模型（如 PaLM、LLaMA 等）中被广泛采用，以平衡模型性能和计算效率。